In [11]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import time

In [20]:
# Code Generation
nr_codewords = int(1e6)
bits_info = torch.randint(0, 2, size=(4, 1, 1000000), dtype=torch.int)


print(bits_info.shape)

torch.Size([4, 1, 1000000])


Hamming(7,4) Encoder

In [23]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.

        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code

        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.int)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        # assert input_data.size(1) == self.generator_matrix.shape[1], "Input data must have same generator matrix row number bits."

        # Perform matrix multiplication to encode the data
        result_tensor = (self.generator_matrix @ input_data.squeeze(1)).unsqueeze(1) % 2

        return result_tensor

In [24]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[[1, 0, 1,  ..., 1, 1, 0]],

        [[1, 1, 1,  ..., 0, 0, 1]],

        [[1, 0, 1,  ..., 0, 0, 0]],

        ...,

        [[0, 1, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 1]],

        [[0, 1, 0,  ..., 1, 1, 0]]], dtype=torch.int32)


BPSK Modulator + Noise

In [25]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

        Args:
            codeword: data received from the Hamming(7,4) encoder(Tensor)

        Returns:
            data: Tensor contain all data modulated and add noise
        """
        super(bpsk_modulator, self).__init__()

    def forward(self, codeword, snr_dB):

        # data = torch.tensor(data, dtype=float)
        data = codeword.to(dtype=torch.float)

        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1

            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(snr_dB / 10))
            noise = torch.sqrt(1/(2*noise_power)) * torch.randn(len(bits))
            noised_signal = bits + noise
            # noised_signal = bits
            data[i] = noised_signal


        return data

In [26]:
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
modulated_noise_signal = modulator(encoded_codeword, snr_dB)
print(modulated_noise_signal)

tensor([[[ 0.9273, -1.0727,  0.9273,  ...,  0.9273,  0.9273, -1.0727]],

        [[ 1.2031,  1.2031,  1.2031,  ..., -0.7969, -0.7969,  1.2031]],

        [[ 1.0634, -0.9366,  1.0634,  ..., -0.9366, -0.9366, -0.9366]],

        ...,

        [[-0.9872,  1.0128, -0.9872,  ..., -0.9872, -0.9872, -0.9872]],

        [[-0.9193, -0.9193, -0.9193,  ..., -0.9193, -0.9193,  1.0807]],

        [[-0.9627,  1.0373, -0.9627,  ...,  1.0373,  1.0373, -0.9627]]])


LLR Log-likelihood
y = s + n
Assuming that \( s \) is equally likely to be 0 or 1, and \( n \) is Gaussian with zero mean and variance \( N_0/2 \), where \( N_0 \) is the noise power spectral density.


In [27]:
def llr(signal, snr):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal from BPSK.
        noise_std (float): Standard deviation of the noise.

    Returns:
        llr: Log Likelihood Ratio (LLR) values.
    """

    # Assuming Binary Phase Shift Keying (BPSK) modulation
    noise_std = torch.sqrt(torch.tensor(10**(snr / 10)))

    # Calculate the LLR
    llr = 2 * signal * noise_std

    # return llr_values, llr
    return llr


In [28]:
llr_output = llr(modulated_noise_signal, snr_dB)
print("LLR values:", llr_output)

LLR values: tensor([[[ 10.4288, -12.0648,  10.4288,  ...,  10.4288,  10.4288, -12.0648]],

        [[ 13.5309,  13.5309,  13.5309,  ...,  -8.9627,  -8.9627,  13.5309]],

        [[ 11.9598, -10.5338,  11.9598,  ..., -10.5338, -10.5338, -10.5338]],

        ...,

        [[-11.1029,  11.3907, -11.1029,  ..., -11.1029, -11.1029, -11.1029]],

        [[-10.3392, -10.3392, -10.3392,  ..., -10.3392, -10.3392,  12.1544]],

        [[-10.8273,  11.6663, -10.8273,  ...,  11.6663,  11.6663, -10.8273]]])


LDPC Decoder

Strange behavior recording:
In 100 7-bit codewords, the speed on MPS(GPU) is slower than CPU. Reason unknown.

In [29]:
class LDPCBeliefPropagation(torch.nn.Module):
    def __init__(self, H):
        """
        LDPC Belief Propagation.

        Args:
            H: Low density parity code for building tanner graph.
            llr: Log Likelihood Ratio (LLR) values. Only for 7-bit codeword.

        Returns:
            estimated_bits: the output result from belief propagation.
        """

        super(LDPCBeliefPropagation, self).__init__()
        self.H = H
        self.num_check_nodes, self.num_variable_nodes = H.shape

        # Initialize messages
        self.messages_v_to_c = torch.ones((self.num_variable_nodes, self.num_check_nodes),dtype=torch.float)
        self.messages_c_to_v = torch.zeros((self.num_check_nodes, self.num_variable_nodes),dtype=torch.float)

    def forward(self, llr, max_iter):
        for iteration in range(max_iter):
            # Variable to check node messages
            for i in range(self.num_variable_nodes):
                for j in range(self.num_check_nodes):
                    # Compute messages from variable to check nodes
                    connected_checks = self.H[j, :] == 1
                    product = torch.prod(torch.tanh(0.5 * self.messages_v_to_c[connected_checks, j]))
                    self.messages_v_to_c[i, j] = torch.sign(llr[i]) * product

            # Check to variable node messages
            for i in range(self.num_check_nodes):
                for j in range(self.num_variable_nodes):
                    # Compute messages from check to variable nodes
                    connected_vars = self.H[:, j] == 1
                    sum_msgs = torch.sum(self.messages_c_to_v[connected_vars, i]) - self.messages_v_to_c[j, i]
                    self.messages_c_to_v[i, j] = 2 * torch.atan(torch.exp(0.5 * sum_msgs))

        # Calculate the final estimated bits and only take first four bits
        estimated_bits = torch.sign(llr) * torch.prod(torch.tanh(0.5 * self.messages_c_to_v), dim=0)
        estimated_bits = torch.where(estimated_bits>0, torch.tensor(1), torch.tensor(0))
        estimated_bits = estimated_bits[0:4]


        return estimated_bits


In [33]:
# Define LDPC parameters
H = torch.tensor([ [1, 1, 1, 0, 0, 0, 0],
                   [0, 0, 1, 1, 1, 0, 0],
                   [0, 1, 0, 0, 1, 1, 0],
                   [1, 0, 0, 1, 0, 0, 1],])
iter = 10
ldpc_bp = LDPCBeliefPropagation(H)
n = 2000 # For n iteration, print time

# Store the final result from LDPC
tensor_size = torch.Size([llr_output.shape[0], 4])
final_result = torch.zeros(tensor_size)

#loop all the llr and get result.

estimated_bits = ldpc_bp(llr_output, iter)
final_result = estimated_bits


if i>0 and i % n == 0:
    print(f"Procedure {i}, Time taken for {n} 7-bit codeword: {iteration_time} seconds")

print(final_result)

RuntimeError: ignored

Comparation and Plot

In [ ]:
def calculate_ber(transmitted_bits, origin_bits):
    # Ensure that both tensors have the same shape
    assert transmitted_bits.shape == origin_bits.shape, "Shapes of transmitted and received bits must be the same."

    # Calculate the bit errors
    errors = (transmitted_bits != origin_bits).sum().item()

    # Calculate the Bit Error Rate (BER)
    ber = errors / transmitted_bits.numel()

    return ber

0.0


In [ ]:
# Describe the data:
# bits_info: original signal
decoded_bits = final_result #output from Maximum Likelihood
# decoded_bits = llr_output # Output from log-likelihood
# decoded_bits =

ber = calculate_ber(decoded_bits, bits_info)
print(ber)